<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [3]:
train = pd.read_csv('flight-delays-2017/flight_delays_train.csv')
test = pd.read_csv('flight-delays-2017/flight_delays_test.csv')

In [5]:
#train
train['DayofMonth'] = train['DayofMonth'].apply(lambda x: int(x.split('-')[1]))
train['Month'] = train['Month'].apply(lambda x: int(x.split('-')[1]))
train['DayOfWeek'] = train['DayOfWeek'].apply(lambda x: int(x.split('-')[1]))
train['Time_hour'] = train['DepTime'].apply(lambda x: int(x/60))
train['Time_day'] = train['DepTime'].apply(lambda x: round(x/60/24, 0))
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Time_hour,Time_day
0,8,21,7,1934,AA,ATL,DFW,732,N,32,1.0
1,4,20,3,1548,US,PIT,MCO,834,N,25,1.0
2,9,2,5,1422,XE,RDU,CLE,416,N,23,1.0
3,11,25,6,1015,OO,DEN,MEM,872,N,16,1.0
4,10,7,6,1828,WN,MDW,OMA,423,Y,30,1.0


In [6]:
#train
test['DayofMonth'] = test['DayofMonth'].apply(lambda x: int(x.split('-')[1]))
test['Month'] = test['Month'].apply(lambda x: int(x.split('-')[1]))
test['DayOfWeek'] = test['DayOfWeek'].apply(lambda x: int(x.split('-')[1]))
test['Time_hour'] = test['DepTime'].apply(lambda x: int(x/60))
test['Time_day'] = test['DepTime'].apply(lambda x: round(x/60/24, 0))
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Time_hour,Time_day
0,7,25,3,615,YV,MRY,PHX,598,10,0.0
1,4,17,2,739,WN,LAS,HOU,1235,12,1.0
2,12,2,7,651,MQ,GSP,ORD,577,10,0.0
3,3,25,7,1614,WN,BWI,MHT,377,26,1.0
4,6,6,3,1505,UA,ORD,STL,258,25,1.0


In [7]:
y = train['dep_delayed_15min'].map({"Y":1, "N":0})
#train = train.drop(['dep_delayed_15min'], axis=1)
y

0        0
1        0
2        0
3        0
4        1
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: dep_delayed_15min, Length: 100000, dtype: int64

In [17]:
def tranform(data):
    for _c in data.select_dtypes(include=['object']).columns:
    
        data[_c]  = pd.Categorical(data[_c])
#data_transformed = pd.get_dummies(tranform(train), drop_first=True)#
#data_transformed.astype(int)
tranform(train)

In [18]:
train

,Month,DayofMonth,DayOfWeek,DepTime,Distance,Time_hour,Time_day
0,8,21,7,1934,732,32,1.0
1,4,20,3,1548,834,25,1.0
2,9,2,5,1422,416,23,1.0
3,11,25,6,1015,872,16,1.0
4,10,7,6,1828,423,30,1.0
...,...,...,...,...,...,...,...
99995,5,4,3,1618,199,26,1.0
99996,1,18,3,804,884,13,1.0
99997,1,24,2,1901,1076,31,1.0
99998,4,27,4,1515,140,25,1.0


In [8]:
train = train.drop(['UniqueCarrier','Origin','Dest'], axis=1)
test = test.drop(['UniqueCarrier','Origin','Dest'], axis=1)

for _c in train.select_dtypes(include=['object']).columns:
  #  print(_c)
    train[_c]  = pd.Categorical(train[_c])
data_train = pd.get_dummies(train, drop_first= True)
data_train


for _c in test.select_dtypes(include=['object']).columns:
  #  print(_c)
    test[_c]  = pd.Categorical(test[_c])
data_test = pd.get_dummies(test, drop_first= True)
data_test

#test.head()

Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

X_train, y_train = data_train.loc[:,:"dep_delayed_15min_Y"].values, data_train['dep_delayed_15min_Y'].values
X_test = data_test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [9]:
train

,Month,DayofMonth,DayOfWeek,DepTime,Distance,Time_hour,Time_day
0,8,21,7,1934,732,32,1.0
1,4,20,3,1548,834,25,1.0
2,9,2,5,1422,416,23,1.0
3,11,25,6,1015,872,16,1.0
4,10,7,6,1828,423,30,1.0
...,...,...,...,...,...,...,...
99995,5,4,3,1618,199,26,1.0
99996,1,18,3,804,884,13,1.0
99997,1,24,2,1901,1076,31,1.0
99998,4,27,4,1515,140,25,1.0


In [10]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(train, 
                                                                y, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [12]:
X_train_part, y_train_part

(       Month  DayofMonth  DayOfWeek  DepTime  Distance  Time_hour  Time_day
 98187      4          18          1     1148       559         19       1.0
 25139      8          13          7      645       612         10       0.0
 83491      4           5          3      525       102          8       0.0
 8295      12          31          7     1336      1249         22       1.0
 65284      8           9          2     1653      1171         27       1.0
 ...      ...         ...        ...      ...       ...        ...       ...
 42297      1          23          1     1845       368         30       1.0
 98710     11          11          5      920       116         15       1.0
 34959      3          25          6      948       187         15       1.0
 64753     10           8          6     1402      1400         23       1.0
 76399     12          13          3      913       293         15       1.0
 
 [70000 rows x 7 columns],
 98187    0
 25139    0
 83491    0
 8295     0

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(df.drop('Churn', axis=1), df['Churn'],
#                                                    test_size=0.3, random_state=42)
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [ ]:
#1
params = {
    'objective':'binary:logistic',
    'max_depth':3,
    'silent':1,
    'eta':0.5
}

num_rounds = 10

In [ ]:
%%time
model = XGBClassifier()
model.fit(X_train_part, y_train_part)

model_valid_pred = model.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, model_valid_pred)

In [ ]:
model.fit(train, y)
model_test_pred = model.predict_proba(test)[:, 1]

pd.Series(model_test_pred, 
          name='dep_delayed_15min').to_csv('logit_2feat.csv', 
                                           index_label='id', 
                                           header=True)

In [ ]:
%%time
#1
params = {
    'objective':'binary:logistic',
    'max_depth':3,
    'silent':1,
    'eta':0.5
}

num_rounds = 10


model = XGBClassifier(params)
model.fit(X_train_part, y_train_part)

model_valid_pred = model.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, model_valid_pred)